In [1]:
import os
import cv2
import networkx as nx

import sys
sys.path.insert(0, '/scratch/autocnet')

from scipy.misc import bytescale

import autocnet
import autocnet.fileio.io_json as io_json

from autocnet.examples import get_path
from autocnet.fileio.io_gdal import GeoDataset
from autocnet.graph.network import CandidateGraph
from autocnet.matcher import feature_extractor as fe
from autocnet.matcher.matcher import FlannMatcher


%pylab qt4

ImportError: No module named 'autocnet'

In [2]:
# View the input json adjacency file
adjacency_file = get_path('two_image_adjacency.json')
# This file is a dictionary of images with a corresponding list of known adjacent images
print(type(io_json.read_json(adjacency_file)))
print( io_json.read_json(adjacency_file))

<class 'dict'>
{'AS15-M-0298_SML.png': ['AS15-M-0297_SML.png'], 'AS15-M-0297_SML.png': ['AS15-M-0298_SML.png']}


In [3]:
# Set up dictionary of images and their serial numbers.
serial_numbers = {'AS15-M-0295_SML.png': 'APOLLO15/METRIC/1971-07-31T01:24:11.754',
                  'AS15-M-0296_SML.png': 'APOLLO15/METRIC/1971-07-31T01:24:36.970',
                  'AS15-M-0297_SML.png': 'APOLLO15/METRIC/1971-07-31T01:25:02.243',
                  'AS15-M-0298_SML.png': 'APOLLO15/METRIC/1971-07-31T01:25:27.457',
                  'AS15-M-0299_SML.png': 'APOLLO15/METRIC/1971-07-31T01:25:52.669',
                  'AS15-M-0300_SML.png': 'APOLLO15/METRIC/1971-07-31T01:26:17.923'}

# Create an adjacency graph from a json file
adjacencyGraph = CandidateGraph.from_adjacency(adjacency_file)
nx.draw(adjacencyGraph)

In [4]:
# Extract image data and attribute nodes

# Get the path without file name
basepath = os.path.dirname(adjacency_file)

# Loop through the nodes (i.e. file names) on the graph and fill in their attributes dictionaries.
# These attributes are...
#      geo dataset file name (as handle)
#      image data (as an NumPy array)
#      features (as keypoints and descriptors)
for node, attributes in adjacencyGraph.nodes_iter(data=True):
    
    attributes['handle'] = GeoDataset(os.path.join(basepath, node))
    attributes['image'] = bytescale(attributes['handle'].read_array())
    attributes['keypoints'], attributes['descriptors'] = fe.extract_features(attributes['image'], 
                                                                             {'nfeatures' : 25})

In [5]:
# Apply a FLANN matcher
matcher = FlannMatcher()

# Loop through the nodes on the graph and feature descriptors to the matcher
for node, attributes in adjacencyGraph.nodes_iter(data=True): 
    matcher.add(attributes['descriptors'], key=node)
    
# build KD-Tree using the feature descriptors
matcher.train()

In [6]:
# Loop through the nodes on the graph to find all features that match at 1 neighbor
# These matches are returned as PANDAS dataframes and added to the adjacency graph
for node, attributes in adjacencyGraph.nodes_iter(data=True):
    descriptors = attributes['descriptors']
    matches = matcher.query(descriptors, 2)
    adjacencyGraph.add_matches(node, matches) 

In [14]:
# rewrite matcher::query method to return matches from knnMatch() call
def demo_get_matches(self, descriptor, k=3, self_neighbor=True):
    idx = 0
    if self_neighbor:
        idx = 1
    return self._flann_matcher.knnMatch(descriptor, k=k)

img = []
for node, attributes in adjacencyGraph.nodes_iter(data=True):
    
    descriptors = attributes['descriptors']
    
    matches = demo_get_matches(matcher, descriptors, 2)
    
    draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = None,
                   flags = 0)
    
    img.append(cv2.drawMatchesKnn(cv2.imread(get_path(adjacencyGraph.nodes()[0])),
                                  adjacencyGraph.node[adjacencyGraph.nodes()[0]]['keypoints'],
                                  cv2.imread(get_path(adjacencyGraph.nodes()[1])),
                                  adjacencyGraph.node[adjacencyGraph.nodes()[1]]['keypoints'],
                                  matches, 
                                  None, 
                                  flags=2))

In [16]:
plt.figure(0)
plt.imshow(img[0])

plt.figure(1)
plt.imshow(img[1])

plt.show()